## pitch2017前処理

#### 7
- 1/4ずつ集計

#### 6
- 選手IDごとのコースの実績

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import feather
pd.set_option('display.max_Columns', 100)

In [2]:
train_pitch = pd.read_feather('data/train_pitch.f')
print(train_pitch.shape)

(257117, 51)


### 1/4ずつデータを分けて集計する

In [3]:
train_pitch_1 = train_pitch.query('index <= 60000')
train_pitch_2 = train_pitch.query('index > 60000 & index <= 120000')
train_pitch_3 = train_pitch.query('index > 120000 & index <= 180000')
train_pitch_4 = train_pitch.query('index > 180000')

### 以下を4回繰り返す

In [81]:
train_pitch = train_pitch_4

In [82]:
OUT_PIT = 'intermediate/pit_2017_7_4.f'
OUT_BAT = 'intermediate/bat_2017_7_4.f'

### 左右

In [83]:
train_pitch.replace('左', 'L', inplace=True)
train_pitch.replace('右', 'R', inplace=True)
train_pitch['pit_bat'] = train_pitch['投手投球左右'] + '_' + train_pitch['打者打席左右']

In [84]:
train_pitch.rename(columns={'球種': 'ball', '投球位置区域': 'course'}, inplace=True)

### 投手
#### 球種
0:ストレート 1:カーブ 2:スライダー 3:シュート 4:フォーク 5:チェンジアップ 6:シンカー 7:カットボール

In [85]:
pitch_ball = train_pitch[['投手ID','pit_bat','ball']]
pitch_ball = pd.get_dummies(pitch_ball, columns=['ball'])

In [86]:
groupby_pit = pitch_ball.groupby(['投手ID','pit_bat']).sum()
groupby_pit.rename(columns={
    'ball_0': 'straight',
    'ball_1': 'curve',
    'ball_2': 'slider',
    'ball_3': 'shoot',
    'ball_4': 'fork',
    'ball_5': 'changeup',
    'ball_6': 'sinker',
    'ball_7': 'cutball',
}, inplace=True)
pitch_ball = groupby_pit.reset_index(inplace=False)

In [87]:
pitch_ball['total'] = (pitch_ball['straight'] + pitch_ball['curve'] + pitch_ball['slider'] + pitch_ball['shoot']
                       + pitch_ball['fork'] + pitch_ball['changeup'] + pitch_ball['sinker'] + pitch_ball['cutball'])

In [88]:
pitch_ball.head()

,投手ID,pit_bat,straight,curve,slider,shoot,fork,changeup,sinker,cutball,total
0,11606,R_L,20.0,10.0,0.0,0.0,15.0,0.0,0.0,4.0,49.0
1,11606,R_R,33.0,28.0,0.0,0.0,14.0,0.0,0.0,0.0,75.0
2,11766,L_L,8.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,16.0
3,11766,L_R,5.0,0.0,2.0,0.0,0.0,1.0,1.0,0.0,9.0
4,11807,R_L,105.0,16.0,1.0,3.0,58.0,0.0,0.0,1.0,184.0


### コース

In [89]:
pitch_course = train_pitch[['投手ID','pit_bat','course']]
pitch_course = pd.get_dummies(pitch_course, columns=['course'])

In [90]:
groupby_course = pitch_course.groupby(['投手ID','pit_bat']).sum()
pitch_course = groupby_course.reset_index(inplace=False)

In [91]:
pitch_course.head()

,投手ID,pit_bat,course_0,course_1,course_2,course_3,course_4,course_5,course_6,course_7,course_8,course_9,course_10,course_11,course_12
0,11606,R_L,2,5,3,0,4,1,0,2,3,4,1,7,17
1,11606,R_R,1,2,2,1,6,3,4,6,5,3,6,10,26
2,11766,L_L,0,2,0,2,2,0,0,3,0,1,3,2,1
3,11766,L_R,1,0,2,1,0,0,0,0,0,0,1,2,2
4,11807,R_L,11,10,12,6,10,8,5,12,13,24,22,25,26


### 登板試合数

In [92]:
pit_game = train_pitch[['投手ID', '試合ID']].groupby(['投手ID', '試合ID']).count()
pit_game = pd.DataFrame(pit_game.groupby(['投手ID']).size())
pit_game.reset_index(inplace=True)
pit_game.rename(columns={0: 'pit_game_cnt'}, inplace=True)

### イニング数

In [93]:
pit_inning = train_pitch[['投手ID', '試合ID', 'イニング']].groupby(['投手ID', '試合ID', 'イニング']).count()
pit_inning = pd.DataFrame(pit_inning.groupby(['投手ID']).size())
pit_inning.reset_index(inplace=True)
pit_inning.rename(columns={0: 'pit_inning_cnt'}, inplace=True)

### 対戦打者数

In [94]:
pit_batcnt = train_pitch[['投手ID', 'pit_bat', '試合ID', 'イニング', 'イニング内打席数']].groupby(['投手ID', 'pit_bat', '試合ID', 'イニング', 'イニング内打席数']).count()
pit_batcnt = pd.DataFrame(pit_batcnt.groupby(['投手ID','pit_bat']).size())
pit_batcnt.reset_index(inplace=True)
pit_batcnt.rename(columns={0: 'pit_batter_cnt'}, inplace=True)

### 投手実績まとめ

In [95]:
pitch_ball = pitch_ball.merge(pitch_course, on=['投手ID','pit_bat'], how='left')
pitch_ball = pitch_ball.merge(pit_game, on='投手ID', how='left')
pitch_ball = pitch_ball.merge(pit_inning, on='投手ID', how='left')
pitch_ball = pitch_ball.merge(pit_batcnt, on=['投手ID','pit_bat'], how='left')

In [96]:
print(pitch_ball.shape)
pitch_ball.head()

(514, 27)


,投手ID,pit_bat,straight,curve,slider,shoot,fork,changeup,sinker,cutball,total,course_0,course_1,course_2,course_3,course_4,course_5,course_6,course_7,course_8,course_9,course_10,course_11,course_12,pit_game_cnt,pit_inning_cnt,pit_batter_cnt
0,11606,R_L,20.0,10.0,0.0,0.0,15.0,0.0,0.0,4.0,49.0,2,5,3,0,4,1,0,2,3,4,1,7,17,7,10,13
1,11606,R_R,33.0,28.0,0.0,0.0,14.0,0.0,0.0,0.0,75.0,1,2,2,1,6,3,4,6,5,3,6,10,26,7,10,18
2,11766,L_L,8.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,16.0,0,2,0,2,2,0,0,3,0,1,3,2,1,3,3,7
3,11766,L_R,5.0,0.0,2.0,0.0,0.0,1.0,1.0,0.0,9.0,1,0,2,1,0,0,0,0,0,0,1,2,2,3,3,3
4,11807,R_L,105.0,16.0,1.0,3.0,58.0,0.0,0.0,1.0,184.0,11,10,12,6,10,8,5,12,13,24,22,25,26,19,23,36


In [97]:
pitch_ball.describe()

,投手ID,straight,curve,slider,shoot,fork,changeup,sinker,cutball,total,course_0,course_1,course_2,course_3,course_4,course_5,course_6,course_7,course_8,course_9,course_10,course_11,course_12,pit_game_cnt,pit_inning_cnt,pit_batter_cnt
count,5.140000e+02,514.000000,514.000000,514.000000,514.000000,514.000000,514.000000,514.000000,514.000000,514.000000,514.000000,514.000000,514.000000,514.000000,514.000000,514.000000,514.000000,514.000000,514.000000,514.000000,514.000000,514.00000,514.000000,514.000000,514.000000,514.000000
mean,1.177323e+06,71.492218,11.165370,27.583658,10.013619,13.406615,8.056420,1.198444,7.114786,150.031128,6.484436,9.750973,8.523346,6.980545,8.819066,7.593385,6.194553,9.513619,8.326848,15.896887,12.904669,20.81323,28.229572,7.840467,19.015564,37.795720
std,4.157446e+05,67.419788,20.222396,32.825931,22.810641,23.424052,19.754999,9.044805,19.309838,131.518212,6.770878,10.353228,9.323953,6.869404,8.454923,7.366488,6.949069,10.082738,9.417446,16.145388,13.897307,22.95160,29.419097,6.280158,15.603011,33.078899
min,1.160600e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,1.000000,1.000000,1.000000
25%,1.000011e+06,20.250000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,48.000000,2.000000,3.000000,2.000000,2.000000,2.000000,2.000000,1.000000,2.000000,2.000000,4.000000,3.000000,5.00000,7.000000,3.000000,6.000000,12.000000
50%,1.300047e+06,51.000000,2.000000,15.000000,0.000000,2.000000,0.000000,0.000000,0.000000,112.000000,4.000000,6.000000,5.000000,5.000000,7.000000,5.000000,4.000000,6.000000,5.000000,11.000000,8.000000,13.00000,18.000000,6.000000,16.000000,29.000000
75%,1.500111e+06,100.750000,14.000000,42.750000,7.000000,18.000000,5.750000,0.000000,1.000000,195.000000,9.000000,14.000000,12.000000,10.000000,12.000000,11.000000,8.000000,13.000000,11.000000,22.000000,17.750000,27.00000,38.750000,12.000000,25.000000,49.750000
max,1.700016e+06,366.000000,130.000000,210.000000,172.000000,155.000000,142.000000,163.000000,149.000000,635.000000,47.000000,58.000000,60.000000,34.000000,45.000000,43.000000,39.000000,52.000000,50.000000,97.000000,86.000000,144.00000,160.000000,26.000000,71.000000,159.000000


In [98]:
pitch_ball.to_feather(OUT_PIT)

## 野手
### 打席数

In [99]:
bat_ball = train_pitch[['打者ID', '試合ID', 'イニング', 'イニング内打席数']].groupby(['打者ID', '試合ID', 'イニング', 'イニング内打席数']).count()
bat_ball = pd.DataFrame(bat_ball.groupby(['打者ID']).size())
bat_ball.reset_index(inplace=True)
bat_ball.rename(columns={0: 'batter_cnt'}, inplace=True)

### 試合数

In [100]:
bat_game = train_pitch[['打者ID', '試合ID']].groupby(['打者ID', '試合ID']).count()
bat_game = pd.DataFrame(bat_game.groupby(['打者ID']).size())
bat_game.reset_index(inplace=True)
bat_game.rename(columns={0: 'bat_game_cnt'}, inplace=True)

### 打者成績まとめ

In [101]:
bat_ball = bat_ball.merge(bat_game, on='打者ID', how='left')

In [102]:
print(bat_ball.shape)
bat_ball.head()

(342, 3)


,打者ID,batter_cnt,bat_game_cnt
0,11270,16,6
1,11302,57,29
2,11343,6,6
3,11436,33,13
4,11511,1,1


In [103]:
bat_ball.describe()

,打者ID,batter_cnt,bat_game_cnt
count,3.420000e+02,342.000000,342.000000
mean,1.034569e+06,56.801170,16.970760
std,4.754627e+05,63.498966,15.021754
min,1.127000e+04,1.000000,1.000000
25%,7.000960e+05,7.000000,4.000000
50%,1.100104e+06,22.000000,10.500000
75%,1.400187e+06,99.750000,30.750000
max,1.700019e+06,206.000000,50.000000


In [104]:
bat_ball.to_feather(OUT_BAT)